In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [21]:
# The directory for top level folder
dir_ = "/home/sugam/Work/20-29 Deep Learning/22 Projects/Optimization of Energy Using AIML"

processed_data_dir = dir_+'/data/Processed/'
model_dir = dir_+'/models/'

In [22]:
# Load your dataset
df = pd.read_pickle(processed_data_dir + "train_df_energy_final.pkl")

In [23]:
df.head()

,load_x,year,month,day,hour,minute,weekday,week_num,is_weekdend,load_min,...,rolling_mean_tmp_1_30,rolling_mean_tmp_1_60,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_30,rolling_mean_tmp_7_60,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_30,rolling_mean_tmp_14_60
0,65.8125,2018,1,1,1,0,0,1,0,38.53125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,65.6875,2018,1,1,1,15,0,1,0,38.53125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,66.0000,2018,1,1,1,30,0,1,0,38.53125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,65.1875,2018,1,1,1,45,0,1,0,38.53125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,70.6875,2018,1,1,2,0,0,1,0,38.53125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df.rename(columns = {"load_x":"load"},inplace=True)

In [25]:
# Split the dataset using fixed split 80% into training and 20% into testing
train_size = int(len(df) * 0.8)
train_data, test_data = df[:train_size], df[train_size:]
target_columns = ["load"] 

X_train, y_train = train_data.drop(target_columns, axis=1), train_data[target_columns]
X_test, y_test = test_data.drop(target_columns, axis=1), test_data[target_columns]

In [26]:
print(f"Shape of Train data: {X_train.shape}")
print(f"Shape of Test data: {X_test.shape}")

Shape of Train data: (82438, 50)
Shape of Test data: (20610, 50)


In [27]:
# Making year as 0, 1,....2 instead of 2018, 2019 , 2020
df["year"] = df["year"]-df["year"].min()

In [28]:
# ss = StandardScaler()
# X_train = ss.fit_transform(X_train)
# X_test = ss.fit_transform(X_test)

In [33]:
X_test.describe().loc[["min","max"],:].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, min to max
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    2 non-null      float64
 1   month                   2 non-null      float64
 2   day                     2 non-null      float64
 3   hour                    2 non-null      float64
 4   minute                  2 non-null      float64
 5   weekday                 2 non-null      float64
 6   week_num                2 non-null      float64
 7   is_weekdend             2 non-null      float64
 8   load_min                2 non-null      float64
 9   load_max                2 non-null      float64
 10  load_mean               2 non-null      float64
 11  load_std                2 non-null      float64
 12  load_norm               2 non-null      float64
 13  load_lag_1              2 non-null      float64
 14  load_lag_2              2 non-null      float64

In [29]:
# Random Forest model
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Calculate RMSE for Random Forest
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
print("Random Forest RMSE:", rf_rmse)

ValueError: Input X contains infinity or a value too large for dtype('float32').

In [15]:
param_grid = {
    'n_estimators': [100, 200, 300],  # You can specify a range of values
    'max_depth': [None, 10, 20, 30],
    # Add more hyperparameters to tune
}


In [16]:
grid = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
best_model = grid.best_estimator_


In [ ]:
rf_predictions = best_model.predict(X_test)


In [ ]:
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
print("Random Forest RMSE:", rf_rmse)

mae = mean_absolute_error(y_test, rf_predictions)
print("Mean Absolute Error (MAE):", mae)

r2 = r2_score(y_test, rf_predictions)
print("R-squared (R^2):", r2)


In [ ]:
print("Best Hyperparameters:", grid.best_params_)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, rf_predictions)
print("Mean Absolute Error (MAE):", mae)

# Calculate R-squared (R^2)
r2 = r2_score(y_test, rf_predictions)
print("R-squared (R^2):", r2)


Mean Absolute Error (MAE): 6.569529586582666
R-squared (R^2): -4.210345324189758
